In [9]:
import numpy as np
import pandas as pd
#from haversine import haversine, Unit
import geopy.distance
from tqdm import tqdm

In [2]:
large_df = pd.read_csv('../DAOU/test_large_out_v2.csv', index_col=0, encoding='euc-kr')
large_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6664 entries, 0 to 6663
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   사업장명                    6664 non-null   object 
 1   업소유형                    874 non-null    object 
 2   업태구분명                   6664 non-null   object 
 3   지번주소                    6653 non-null   object 
 4   도로명주소                   3986 non-null   object 
 5   위도                      5774 non-null   float64
 6   경도                      5774 non-null   float64
 7   별점                      310 non-null    float64
 8   방문자 리뷰수                 685 non-null    object 
 9   블로그 리뷰수                 513 non-null    object 
 10  지하철역과의 거리               774 non-null    object 
 11  도보시간                    94 non-null     object 
 12  한실수                     5434 non-null   float64
 13  양실수                     5347 non-null   float64
 14  욕실수                     4813 non-null   

In [5]:
subway = pd.read_csv('../DABA/서울시 역사마스터 정보.csv', encoding='euc-kr')
subway

,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,127.193877,37.560927
1,9995,강일,5호선,127.175930,37.557490
2,4929,김포공항,김포골드라인,126.801868,37.562360
3,4928,고촌,김포골드라인,126.770345,37.601243
4,4927,풍무,김포골드라인,126.732387,37.612488
...,...,...,...,...,...
760,154,종로5가,1호선,127.001849,37.570926
761,153,종로3가,1호선,126.991847,37.570406
762,152,종각,1호선,126.982923,37.570161
763,151,시청,1호선,126.977088,37.565715


In [6]:
# 프라이머리 키 생성
subway['subway_pk'] = np.nan
for i in range(subway.shape[0]):
    #print(subway['역사명'][i], subway['호선'][i])
    subway['subway_pk'][i] = f"{subway['역사명'][i]}, {subway['호선'][i]}"  

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_5272\2159356481.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subway['subway_pk'][i] = f"{subway['역사명'][i]}, {subway['호선'][i]}"


In [7]:
subway

,역사_ID,역사명,호선,위도,경도,subway_pk
0,9996,미사,5호선,127.193877,37.560927,"미사, 5호선"
1,9995,강일,5호선,127.175930,37.557490,"강일, 5호선"
2,4929,김포공항,김포골드라인,126.801868,37.562360,"김포공항, 김포골드라인"
3,4928,고촌,김포골드라인,126.770345,37.601243,"고촌, 김포골드라인"
4,4927,풍무,김포골드라인,126.732387,37.612488,"풍무, 김포골드라인"
...,...,...,...,...,...,...
760,154,종로5가,1호선,127.001849,37.570926,"종로5가, 1호선"
761,153,종로3가,1호선,126.991847,37.570406,"종로3가, 1호선"
762,152,종각,1호선,126.982923,37.570161,"종각, 1호선"
763,151,시청,1호선,126.977088,37.565715,"시청, 1호선"


In [8]:
large_df.iloc[0]['사업장명']

print(np.nan)

nan


In [10]:
# .. 가까운 지하철 거리 및 도보 시간 구하기
# 가까운 지하철 구하기

large_dist = []

for i in tqdm(range(large_df.shape[0])):
#for i in range(1):
    dummy = []

    if np.isnan(large_df.iloc[i]['위도']) :

        dummy.append(np.nan)
        find_min_j = np.nan
        find_stn_nam = np.nan
        find_stn_lat = np.nan
        find_stn_lon = np.nan
        nearest_stn = np.nan
        time_to_near = np.nan

    else:

        for j in range(subway.shape[0]):

            hotel_latlon = (large_df.iloc[i]['위도'], large_df.iloc[i]['경도'])
            subway_latlon = (subway.iloc[j]['경도'], subway.iloc[j]['위도'])    # 원본데이터에서 위경도 컬럼명 바뀌어 있음

            dist = geopy.distance.geodesic(hotel_latlon, subway_latlon).m
            dummy.append(dist)

        find_min_j = np.argmin(dummy)
        dist_to_stn = np.nanmin(dummy)
        time_to_stn = np.nanmin(dummy)/66.7

        #find_stn_nam = subway.iloc[find_min_j]['역사명']
        find_stn_nam = subway.iloc[find_min_j]['subway_pk']
        find_stn_lat = subway.iloc[find_min_j]['위도']
        find_stn_lon = subway.iloc[find_min_j]['경도']

        
            
        #print(f"호텔:{large_df.iloc[i]['사업장명']} {large_df.iloc[i]['지번주소']} {min(dummy)}m")
    large_dist.append([large_df.iloc[i]['사업장명'],
                        large_df.iloc[i]['지번주소'],
                        large_df.iloc[i]['위도'],
                        large_df.iloc[i]['경도'],
                        find_stn_nam,
                        find_stn_lat,
                        find_stn_lon,
                        dist_to_stn,
                        time_to_stn]) # 시속 4km --> 분속 66.7m



100%|██████████| 6664/6664 [34:45<00:00,  3.20it/s]


In [11]:
find_large_dist = pd.DataFrame(large_dist, columns=['사업장명','지번주소','호텔 위도', '호텔 경도','역이름', '역 위도','역 경도', '거리','도보시간(분)'])

#for i in range(10):
#    print(f"호텔:{large_df.iloc[i]['사업장명']}, {df.iloc[i]['거리']}")
find_large_dist.info()
find_large_dist

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6664 entries, 0 to 6663
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   사업장명     6664 non-null   object 
 1   지번주소     6653 non-null   object 
 2   호텔 위도    5774 non-null   float64
 3   호텔 경도    5774 non-null   float64
 4   역이름      5774 non-null   object 
 5   역 위도     5774 non-null   float64
 6   역 경도     5774 non-null   float64
 7   거리       6664 non-null   float64
 8   도보시간(분)  6664 non-null   float64
dtypes: float64(6), object(3)
memory usage: 468.7+ KB


,사업장명,지번주소,호텔 위도,호텔 경도,역이름,역 위도,역 경도,거리,도보시간(분)
0,어반스테이 명동,서울특별시 중구 인현동1가 158,37.563358,126.993360,"충무로, 3호선",126.994072,37.561430,223.010258,3.343482
1,오요5(oYo5),서울특별시 중구 남산동2가 35-2,37.558934,126.984838,"명동, 4호선",126.986325,37.560989,263.215774,3.946263
2,더스테이클래식호텔,서울특별시 중구 남대문로3가 94,37.561480,126.978525,"회현(남대문시장), 4호선",126.978246,37.558514,330.151983,4.949805
3,오요5롯지A(oYo5LodgeA),서울특별시 중구 남산동2가 33-3 1~2층,37.559108,126.984981,"명동, 4호선",126.986325,37.560989,240.223310,3.601549
4,오요5롯지B(oYo5LodgeB),"서울특별시 중구 남산동2가 33 1,2층",37.559222,126.985041,"명동, 4호선",126.986325,37.560989,226.602301,3.397336
...,...,...,...,...,...,...,...,...,...
6659,이녹(李綠),서울특별시 종로구 종로5가 66-1,37.571482,127.002656,"종로5가, 1호선",127.001849,37.570926,94.265899,1.413282
6660,설레임모텔,서울특별시 강서구 화곡동 24-77,37.546468,126.846581,"화곡, 5호선",126.840461,37.541513,771.321547,11.564041
6661,신라장,서울특별시 동작구 신대방동 687-8,37.488098,126.908444,"신대방, 2호선",126.913149,37.487462,422.041104,6.327453
6662,웨이브,서울특별시 성북구 정릉동 408-11,37.606978,127.010455,"정릉, 우이신설선",127.013396,37.603133,499.540547,7.489364


In [12]:
large_df['가까운 지하철역'] = find_large_dist['역이름']
large_df['지하철역과의 거리'] = find_large_dist['거리']
large_df['도보시간'] = find_large_dist['도보시간(분)']

In [13]:
large_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6664 entries, 0 to 6663
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   사업장명                    6664 non-null   object 
 1   업소유형                    874 non-null    object 
 2   업태구분명                   6664 non-null   object 
 3   지번주소                    6653 non-null   object 
 4   도로명주소                   3986 non-null   object 
 5   위도                      5774 non-null   float64
 6   경도                      5774 non-null   float64
 7   별점                      310 non-null    float64
 8   방문자 리뷰수                 685 non-null    object 
 9   블로그 리뷰수                 513 non-null    object 
 10  지하철역과의 거리               6664 non-null   float64
 11  도보시간                    6664 non-null   float64
 12  한실수                     5434 non-null   float64
 13  양실수                     5347 non-null   float64
 14  욕실수                     4813 non-null   

In [14]:
colum_names2 = ['사업장명','업소유형','업태구분명','지번주소','도로명주소','가까운 지하철역', '지하철역과의 거리', '도보시간',
                '위도','경도', '별점','방문자 리뷰수', '블로그 리뷰수', '한실수','양실수','욕실수','구비시설',
                '네이버 이런점이 좋아요 총합', '네이버 이런점이 좋아요 {항목:좋아요수}', '데이터갱신일자',
                '인허가일자','인허가취소일자','영업상태코드','폐업일자','휴업시작일자','휴업종료일자','재개업일자']  # 27

large_df = large_df[colum_names2]
large_df

,사업장명,업소유형,업태구분명,지번주소,도로명주소,가까운 지하철역,지하철역과의 거리,도보시간,위도,경도,...,네이버 이런점이 좋아요 총합,네이버 이런점이 좋아요 {항목:좋아요수},데이터갱신일자,인허가일자,인허가취소일자,영업상태코드,폐업일자,휴업시작일자,휴업종료일자,재개업일자
0,어반스테이 명동,레지던스,숙박업(생활),서울특별시 중구 인현동1가 158,"서울특별시 중구 충무로 24, 3~17층 (인현동1가)","충무로, 3호선",223.010258,3.343482,37.563358,126.993360,...,41.0,"{'깨끗해요': 29},{'조용히 쉬기 좋아요': 22},{'냉난방이 잘돼요': 2...",2022-12-03 00:03:00.0,2021-01-15,NaN,1,NaN,NaN,NaN,NaN
1,오요5(oYo5),호스텔,관광호텔,서울특별시 중구 남산동2가 35-2,"서울특별시 중구 퇴계로18길 43, 지층, 1~2층 (남산동2가)","명동, 4호선",263.215774,3.946263,37.558934,126.984838,...,NaN,NaN,2022-12-03 00:03:00.0,2019-08-02,NaN,1,NaN,NaN,NaN,NaN
2,더스테이클래식호텔,호텔,숙박업(생활),서울특별시 중구 남대문로3가 94,서울특별시 중구 남대문로 27 (남대문로3가),"회현(남대문시장), 4호선",330.151983,4.949805,37.561480,126.978525,...,NaN,NaN,2022-12-02 23:05:00.0,2023-03-13,NaN,1,NaN,NaN,NaN,NaN
3,오요5롯지A(oYo5LodgeA),NaN,숙박업 기타,서울특별시 중구 남산동2가 33-3 1~2층,"서울특별시 중구 퇴계로18길 41 (남산동2가, 1~2층)","명동, 4호선",240.223310,3.601549,37.559108,126.984981,...,NaN,NaN,2022-12-03 00:03:00.0,2017-04-26,NaN,1,NaN,NaN,NaN,NaN
4,오요5롯지B(oYo5LodgeB),NaN,관광호텔,"서울특별시 중구 남산동2가 33 1,2층","서울특별시 중구 퇴계로18길 37 (남산동2가, 1,2층)","명동, 4호선",226.602301,3.397336,37.559222,126.985041,...,NaN,NaN,2022-12-03 00:03:00.0,2017-04-26,NaN,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6659,이녹(李綠),NaN,여관업,서울특별시 종로구 종로5가 66-1,서울특별시 종로구 대학로2길 12 (종로5가),"종로5가, 1호선",94.265899,1.413282,37.571482,127.002656,...,NaN,NaN,2021-12-04 23:04:00.0,19760209,NaN,1,NaN,NaN,NaN,NaN
6660,설레임모텔,모텔,여관업,서울특별시 강서구 화곡동 24-77,서울특별시 강서구 화곡로42길 17 (화곡동),"화곡, 5호선",771.321547,11.564041,37.546468,126.846581,...,NaN,NaN,2021-12-05 22:04:00.0,19970702,NaN,1,NaN,NaN,NaN,NaN
6661,신라장,NaN,여관업,서울특별시 동작구 신대방동 687-8,서울특별시 동작구 신대방3길 59 (신대방동),"신대방, 2호선",422.041104,6.327453,37.488098,126.908444,...,NaN,NaN,2021-12-08 23:08:00.0,19911122,NaN,1,NaN,NaN,NaN,NaN
6662,웨이브,모텔,여관업,서울특별시 성북구 정릉동 408-11,서울특별시 성북구 보국문로 27 (정릉동),"정릉, 우이신설선",499.540547,7.489364,37.606978,127.010455,...,NaN,NaN,2021-12-05 22:09:00.0,20020918,NaN,1,NaN,NaN,NaN,NaN


In [15]:
output_path = "../DAOU/test_large_out_v3.csv"
large_df.to_csv(output_path, encoding='euc-kr')

In [16]:
large_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6664 entries, 0 to 6663
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   사업장명                    6664 non-null   object 
 1   업소유형                    874 non-null    object 
 2   업태구분명                   6664 non-null   object 
 3   지번주소                    6653 non-null   object 
 4   도로명주소                   3986 non-null   object 
 5   가까운 지하철역                5774 non-null   object 
 6   지하철역과의 거리               6664 non-null   float64
 7   도보시간                    6664 non-null   float64
 8   위도                      5774 non-null   float64
 9   경도                      5774 non-null   float64
 10  별점                      310 non-null    float64
 11  방문자 리뷰수                 685 non-null    object 
 12  블로그 리뷰수                 513 non-null    object 
 13  한실수                     5434 non-null   float64
 14  양실수                     5347 non-null   